In [ ]:
# รวมไฟล์
# นำเข้า lib
import config 
import tableauserverclient as TSC
import pyodbc
import schedule
import time

tableau_auth = TSC.PersonalAccessTokenAuth(config.tableau['TOKEN_NAME'], config.tableau['TOKEN_VALUE'], site_id=config.tableau['site'])
server = TSC.Server(config.tableau['server_url'], use_server_version=True)

# connection sql server ที่จะ save log
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={config.sql_server_connection['server']};'
    f'DATABASE={config.sql_server_connection['database']};'
    f'UID={config.sql_server_connection['username']};'
    f'PWD={config.sql_server_connection['password']}'
)
# (1) Main Get Datasource 

# Function Get all Datasources in all Workbooks
def get_embed_datasource():
    with server.auth.sign_in(tableau_auth):
        page_size = 100
        req_option = TSC.RequestOptions(pagesize=page_size)
        all_workbooks = []
        datasource_id = []
        workbook_id = []
        old_data = []
        index = 1
        workbook_name = []
        datasource_name = []
        
        while True:
            workbooks, pagination_item = server.workbooks.get(req_option)
            all_workbooks.extend(workbooks)
            
            if pagination_item.total_available > len(all_workbooks):
                req_option.pagenumber += 1
            else:
                break
        
        print('Searching datasource in Workboook')
        try:    
            for i in all_workbooks:
                workbook = server.workbooks.get_by_id(i.id)
                server.workbooks.populate_connections(workbook)

                for connection in workbook.connections:
                    if connection.embed_password is False:
                        if connection.connection_type == 'sqlserver':
                            if workbook.project_name is not None:
                                # หา extract 
                                datasource = server.datasources.get_by_id(connection.datasource_id)
                                server.datasources.populate_connections(datasource)
                                if datasource.has_extracts is False:
                                    if connection.server_address.lower() in config.panda_connection['server']: 
                                        print(f"Datasource Number: {index}")
                                        print(f"Workbook ID: {workbook.id}")
                                        print(f"Workbook name: {workbook.name}")
                                        print(f"Project ID: {workbook.project_id}")
                                        print(f"Project name: {workbook.project_name}")
                                        print(f"id: {connection.id}")
                                        print(f"Datasource ID: {connection.datasource_id}")
                                        print(f"Datasource name: {connection.datasource_name}")
                                        print(f"Type: {connection.connection_type}")
                                        print(f"Server Address: {connection.server_address}")
                                        print(f"Embed Password: {connection.embed_password}")
                                        print(f"Has Extracts: {datasource.has_extracts}")
                                        print('---------------------------------------')
                                        workbook_id.append(workbook.id)
                                        datasource_id.append(connection.datasource_id)
                                        old_data.append(f"<Old Data = Datasource Number: {index}, Workbook ID: '{workbook.id}', Workbook name: '{workbook.name}', Project ID: '{workbook.project_id}', Project name: '{workbook.project_name}', id: '{connection.id}', Datasource ID: '{connection.datasource_id}', Datasource name: '{connection.datasource_name}',  Type: '{connection.connection_type}', Server Address: '{connection.server_address}', Embed Status: '{connection.embed_password}', Embed Password: '{connection.embed_password}', Has Extracts: '{datasource.has_extracts}'>")
                                        workbook_name.append(workbook.name)
                                        datasource_name.append(connection.datasource_name)
                                        index+=1
                                    else:
                                        pass
                                else:
                                    pass
                            else:
                                pass

                        else:
                            pass   
                    else:
                        pass
            print(f"Total Datasource is {index-1}")
        except ValueError as e:
            print("Finish Get All Datasource (1)")
            print(f"Function Get Datasource from Workbook")
            print(f"Datasource Number: {index}")
            print(f"Workbook ID: {workbook.id}")
            print(f"Workbook name: {workbook.name}")
            print(f"⚠️ Error fetching Datasource ID {connection.datasource_id}: {e}")
            print(f"Datasource name: {connection.datasource_name}")
        except Exception as e:
            print("Finish Get All Datasource (1)")
            print(f"Function Get Datasource from Workbook")
            print(f"Datasource Number: {index}")
            print(f"Workbook ID: {workbook.id}")
            print(f"Workbook name: {workbook.name}")
            print(f"⚠️ Unexpected Error for Datasource ID {connection.datasource_id}: {e}")
            print(f"Datasource name: {connection.datasource_name}")
            print('---------------------------------------')

    # ออกจากระบบ
    print("Finish Get All Datasource (1)")
    server.auth.sign_out()
    return workbook_id,datasource_id,old_data,workbook_name,datasource_name



# (2) Change Embed Password Status เอา datasource_id จาก (1) มา Update

def change_embed_status(datasource_id):
    with server.auth.sign_in(tableau_auth):
        dt = server.datasources.get_by_id(datasource_id)
        server.datasources.populate_connections(dt)
        connection = dt.connections[0]

        connection.server_address = connection.server_address
        connection.username = connection.username
        connection.password = config.panda_connection['password']
        connection.embed_password = config.panda_connection['embed_password']
        connection.embed_password = True

        server.datasources.update_connection(dt, connection)
    
    print(f"Updated connection: {connection.server_address}:{connection.server_port} with username {connection.username} and {connection.embed_password}")



# (3) Check status after update จะเอาค่า workbook_id และ datasource_id จาก Function(1) มาค้นหาใหม่เพราะเผื่อไม่เกิดการอัพเดตจริง 
def check_datasource(workbook_id,datasource_id):
    with server.auth.sign_in(tableau_auth):
        # Get Workbook Information
        try:
            workbook_check = server.workbooks.get_by_id(workbook_id)
            server.workbooks.populate_connections(workbook_check)
            workbook_connection = workbook_check.connections[0]

            # Get Datasource Information
            datasource_check = server.datasources.get_by_id(datasource_id)
            server.datasources.populate_connections(datasource_check)
            datasource_connection = datasource_check.connections[0]

            new_status = workbook_connection.embed_password
            
            workbook_name = workbook_check.name
            datasource_name = workbook_connection.datasource_name
            print('---------------------------------------')

        except ValueError as e:
            print(f"Function Check Datasource from Workbook")
            print(f"Workbook ID: {workbook_id}")
            print(f"Workbook name: {workbook_check.name}")
            print(f"Datasource ID: {workbook_connection.datasource_id}")
            print(f"Datasource name: {workbook_connection.datasource_name}")
            print(e)
            print('---------------------------------------')
        except Exception as e:
            print(f"Function Check Datasource from Workbook")
            print(f"Workbook ID: {workbook_id}")
            print(f"Workbook name: {workbook_check.name}")
            print(f"Datasource ID: {workbook_connection.datasource_id}")
            print(f"Datasource name: {workbook_connection.datasource_name}")
            print(e)
            print('---------------------------------------')
        
    server.auth.sign_out()
    return new_status
    

# (4) Insert log to SQL server
def insert_data(workbook_name,datasource_name,status,detail):
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # insert data
    insert_query = '''
    INSERT INTO [da_dev].[Tableau_Change_Embed_Logs] (workbook_name, datasource_name, status, detail)
    VALUES (?, ?, ?,?)
    '''
    data = (workbook_name, datasource_name, status,detail)
    # Execute the insert statement
    cursor.execute(insert_query, data)

    # Commit the transaction
    conn.commit()
    print(f"Insert Data Workbook Name:{workbook_name}\nData Source: {datasource_name}\nStatus: {status}\nDetail:{detail}")

    # Close the cursor and connection
    cursor.close()
    conn.close()


# เริ่มทำงาน

def job():
    # (1) Main Get Datasource 
    # Function Get all Datasources in all Workbooks
    # รันทุก datasource ที่มีปัญหาให้เปิด comment บรรทัดนี้แล้ว ปิดอีก 4 ด้านล่าง 
    # workbook_id,datasource_id,old_data,workbook_name,datasource_name = get_embed_datasource()
    
    workbook_id=['355d8077-28a5-470c-b8a6-639f779da6f7']
    datasource_id=['8c5f4164-a915-4ee8-bcf9-66d1794640da']
    workbook_name = ['test_embed_password(Pep)']
    datasource_name = ['Test_embeded_password']
    old_data = ['Test_data1']

    # save log to computer
    f = open("embed_change_log.txt", "a")

    print("Job executed")
    for check_workbook_id,check_datasource_id,check_old_data,check_workbook_name,check_datasource_name in zip(workbook_id,datasource_id,old_data,workbook_name,datasource_name):
        # (2) Change Embed Password Status
        change_embed_status(check_datasource_id)
        # (3) Check status after update
        new_status = check_datasource(check_workbook_id,check_datasource_id)
        # (4) Insert log to SQL server
        insert_data(check_workbook_name,check_datasource_name,new_status,check_old_data)
        f.write(f"Workbook ID:{check_workbook_id}, Workbook Name: {check_workbook_name}, Datasource ID: {check_datasource_id}, Datasource Name: {check_datasource_name}, New Status: {new_status}")
    f.close()
# เริ่มทำงาน
# Schedule the job every Monday at 15:33
schedule.every().monday.at("15:33").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

Job executed
Updated connection: sia09-icpr01.sihmis.si:None with username userTableau and True
---------------------------------------
Insert Data Workbook Name:test_embed_password(Pep)
Data Source: Test_embeded_password
Status: True
Detail:Test_data1


KeyboardInterrupt: 